In [1]:
from datetime import date
from pathlib import Path
import pandas as pd
import typing
import math
import sys
import os

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[1])

print(parent_dir)

# Add to sys.path
# sys.path.insert(0, parent_dir)
sys.path.append(parent_dir)



from HedgeModel.MktData.equity_prices import EquityPrices
from HedgeModel.MktData.equity_vol import EquityVol
from HedgeModel.MktData.mkt_data import MktData

from crossasset.crossasset_base import CrossAssetContext
from nxpy import pro

pd.options.display.float_format = "{:,.2f}".format

%load_ext autoreload
%autoreload 2

\\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code


In [38]:
print(f'Base Path:                {str(Path().resolve())}')
print(f'Base Path Parent[0] Path: {str(Path().resolve().parents[0])}')
print(f'Base Path Parent[1] Path: {str(Path().resolve().parents[1])}')
print(f'Base Path Parent[2] Path: {str(Path().resolve().parents[2])}')

Base Path:                \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\prototypes\winterfell
Base Path Parent[0] Path: \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code\prototypes
Base Path Parent[1] Path: \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\1_Code
Base Path Parent[2] Path: \\rgare.net\stlfinmkts\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process


'\\\\rgare.net\\stlfinmkts\\MarketRiskMgmt\\Pricing Requests\\2024-Orion - IUL Hedging\\RGA_Process\\1_Code\\prototypes'

In [2]:
fund_map_file = r'C:\Users\S0053071\Repos\Winterfell\Static_Assumptions\Winterfell_LiabSeg_FundMap.csv'
liab_seg_file = r'C:\Users\S0053071\Repos\Winterfell\Positions\20241231\TG_PCS_Combined_20241231_RGA.csv'

In [28]:
# Get the data
fund_map_df = pd.read_csv(fund_map_file)
liab_seg_df = pd.read_csv(liab_seg_file)

# Remove Buffer -- Keep Term
fund_map_df = fund_map_df[list(fund_map_df.columns)[:-1]]

In [4]:
liab_seg_df.columns

Index(['Segment ID', 'Fund Name', 'Maturity Date', 'Number of Units',
       'IUL EFLIC', 'MSO EFLIC', 'MSO EFLOA', 'IUL EFLOA', 'IUL Reinsured',
       'INDEX Closing Price', 'Remaining Segment Length', 'Strike ATM Call',
       'Strike OTM Call', 'Strike OTM Put', 'Cap', 'Buffer'],
      dtype='object')

In [5]:
fund_map_df.columns

Index(['Product', 'Series', 'Fund_ID', 'Fund_Name', 'Bbg_Idx', 'Opt_Type'], dtype='object')

In [46]:
liab_seg_df

,Segment ID,Fund Name,Maturity Date,Number of Units,IUL EFLIC,MSO EFLIC,MSO EFLOA,IUL EFLOA,IUL Reinsured,INDEX Closing Price,Remaining Segment Length,Strike ATM Call,Strike OTM Call,Strike OTM Put,Cap,Buffer
0,2025011706001XXXXXX,S&P 500 1 Year -25% Buffer,1/17/2025,"281,512.96",0.00,0.75,0.25,0.00,0.00,"5,881.63",0.05,"4,839.81","5,396.39","3,629.86",0.12,0.25
1,2025022106001XXXXXX,S&P 500 1 Year -25% Buffer,2/21/2025,"185,537.82",0.00,0.73,0.27,0.00,0.00,"5,881.63",0.14,"5,005.57","5,618.75","3,754.18",0.12,0.25
2,2025032106001XXXXXX,S&P 500 1 Year -25% Buffer,3/21/2025,"396,632.77",0.00,0.78,0.22,0.00,0.00,"5,881.63",0.22,"5,117.09","5,756.73","3,837.82",0.12,0.25
3,2025041706001XXXXXX,S&P 500 1 Year -25% Buffer,4/17/2025,"184,848.89",0.00,0.72,0.28,0.00,0.00,"5,881.63",0.29,"4,967.23","5,588.13","3,725.42",0.12,0.25
4,2025051606001XXXXXX,S&P 500 1 Year -25% Buffer,5/16/2025,"183,423.75",0.00,0.80,0.20,0.00,0.00,"5,881.63",0.37,"5,303.27","5,939.66","3,977.45",0.12,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,2025081406381XXXXXX,S&P 500 1 Year -100% Buffer,8/14/2025,"172,385.46",0.23,0.00,0.00,0.77,0.00,"5,881.63",0.62,"5,543.22","5,903.53",0.00,0.07,1.00
705,2025091506381XXXXXX,S&P 500 1 Year -100% Buffer,9/15/2025,"179,113.58",0.25,0.00,0.00,0.75,0.00,"5,881.63",0.71,"5,634.58","6,000.83",0.00,0.07,1.00
706,2025101406381XXXXXX,S&P 500 1 Year -100% Buffer,10/14/2025,"168,550.80",0.26,0.00,0.00,0.74,0.00,"5,881.63",0.79,"5,815.26","6,193.25",0.00,0.07,1.00
707,2025111406381XXXXXX,S&P 500 1 Year -100% Buffer,11/14/2025,"189,880.04",0.33,0.00,0.00,0.67,0.00,"5,881.63",0.87,"5,870.62","6,252.21",0.00,0.07,1.00


In [29]:
liab_seg_df['Segment ID'] = liab_seg_df['Segment ID'].str.replace('XXXXXX', '')
liab_seg_df['Fund_ID'] = liab_seg_df['Segment ID'].str[-4:].astype(int)
liab_seg_df['Notional'] = liab_seg_df['Number of Units'] * 100.0
# liab_seg_df['Part'] = 1.0
liab_seg_df['IdxLvl_StartDt'] = liab_seg_df['Strike ATM Call']
liab_seg_df['Contracts'] = liab_seg_df['Notional'] / liab_seg_df['IdxLvl_StartDt']

# Set 'Coins' = 0.75 for this?
liab_seg_df['Coins'] = 0.75


In [30]:
liab_seg_df

,Segment ID,Fund Name,Maturity Date,Number of Units,IUL EFLIC,MSO EFLIC,MSO EFLOA,IUL EFLOA,IUL Reinsured,INDEX Closing Price,Remaining Segment Length,Strike ATM Call,Strike OTM Call,Strike OTM Put,Cap,Buffer,Fund_ID,Notional,IdxLvl_StartDt,Contracts
0,2025011706001,S&P 500 1 Year -25% Buffer,1/17/2025,"281,512.96",0.00,0.75,0.25,0.00,0.00,"5,881.63",0.05,"4,839.81","5,396.39","3,629.86",0.12,0.25,6001,"28,151,296.21","4,839.81","5,816.61"
1,2025022106001,S&P 500 1 Year -25% Buffer,2/21/2025,"185,537.82",0.00,0.73,0.27,0.00,0.00,"5,881.63",0.14,"5,005.57","5,618.75","3,754.18",0.12,0.25,6001,"18,553,781.66","5,005.57","3,706.63"
2,2025032106001,S&P 500 1 Year -25% Buffer,3/21/2025,"396,632.77",0.00,0.78,0.22,0.00,0.00,"5,881.63",0.22,"5,117.09","5,756.73","3,837.82",0.12,0.25,6001,"39,663,277.22","5,117.09","7,751.14"
3,2025041706001,S&P 500 1 Year -25% Buffer,4/17/2025,"184,848.89",0.00,0.72,0.28,0.00,0.00,"5,881.63",0.29,"4,967.23","5,588.13","3,725.42",0.12,0.25,6001,"18,484,888.80","4,967.23","3,721.37"
4,2025051606001,S&P 500 1 Year -25% Buffer,5/16/2025,"183,423.75",0.00,0.80,0.20,0.00,0.00,"5,881.63",0.37,"5,303.27","5,939.66","3,977.45",0.12,0.25,6001,"18,342,375.48","5,303.27","3,458.69"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,2025081406381,S&P 500 1 Year -100% Buffer,8/14/2025,"172,385.46",0.23,0.00,0.00,0.77,0.00,"5,881.63",0.62,"5,543.22","5,903.53",0.00,0.07,1.00,6381,"17,238,546.31","5,543.22","3,109.84"
705,2025091506381,S&P 500 1 Year -100% Buffer,9/15/2025,"179,113.58",0.25,0.00,0.00,0.75,0.00,"5,881.63",0.71,"5,634.58","6,000.83",0.00,0.07,1.00,6381,"17,911,357.93","5,634.58","3,178.83"
706,2025101406381,S&P 500 1 Year -100% Buffer,10/14/2025,"168,550.80",0.26,0.00,0.00,0.74,0.00,"5,881.63",0.79,"5,815.26","6,193.25",0.00,0.07,1.00,6381,"16,855,080.31","5,815.26","2,898.42"
707,2025111406381,S&P 500 1 Year -100% Buffer,11/14/2025,"189,880.04",0.33,0.00,0.00,0.67,0.00,"5,881.63",0.87,"5,870.62","6,252.21",0.00,0.07,1.00,6381,"18,988,003.54","5,870.62","3,234.41"


In [17]:
liab_seg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Segment ID                709 non-null    object 
 1   Fund Name                 709 non-null    object 
 2   Maturity Date             709 non-null    object 
 3   Number of Units           709 non-null    float64
 4   IUL EFLIC                 709 non-null    float64
 5   MSO EFLIC                 709 non-null    float64
 6   MSO EFLOA                 709 non-null    float64
 7   IUL EFLOA                 709 non-null    float64
 8   IUL Reinsured             709 non-null    float64
 9   INDEX Closing Price       709 non-null    float64
 10  Remaining Segment Length  709 non-null    float64
 11  Strike ATM Call           709 non-null    float64
 12  Strike OTM Call           709 non-null    float64
 13  Strike OTM Put            709 non-null    float64
 14  Cap       

In [7]:
liab_seg_df.columns

Index(['Segment ID', 'Fund Name', 'Maturity Date', 'Number of Units',
       'IUL EFLIC', 'MSO EFLIC', 'MSO EFLOA', 'IUL EFLOA', 'IUL Reinsured',
       'INDEX Closing Price', 'Remaining Segment Length', 'Strike ATM Call',
       'Strike OTM Call', 'Strike OTM Put', 'Cap', 'Buffer', 'Fund_ID',
       'Notional', 'Part', 'IdxLvl_StartDt'],
      dtype='object')

In [31]:
# liab_cols_pre_merge = ['Fund_ID', 'Maturity Date', 'Notional', 'Contracts', 'IUL EFLIC', 'IUL EFLOA', 'IUL Reinsured', 'MSO EFLIC', 'MSO EFLOA', 'Cap', 'Buffer', 'Part']
liab_cols_pre_merge = ['Fund_ID', 'Maturity Date', 'Notional', 'Contracts', 'IdxLvl_StartDt', 'IUL EFLIC', 'IUL EFLOA', 'IUL Reinsured', 'MSO EFLIC', 'MSO EFLOA', 'Cap', 'Buffer']
liab_seg_df = liab_seg_df[liab_cols_pre_merge]
liab_seg_df

,Fund_ID,Maturity Date,Notional,Contracts,IdxLvl_StartDt,IUL EFLIC,IUL EFLOA,IUL Reinsured,MSO EFLIC,MSO EFLOA,Cap,Buffer
0,6001,1/17/2025,"28,151,296.21","5,816.61","4,839.81",0.00,0.00,0.00,0.75,0.25,0.12,0.25
1,6001,2/21/2025,"18,553,781.66","3,706.63","5,005.57",0.00,0.00,0.00,0.73,0.27,0.12,0.25
2,6001,3/21/2025,"39,663,277.22","7,751.14","5,117.09",0.00,0.00,0.00,0.78,0.22,0.12,0.25
3,6001,4/17/2025,"18,484,888.80","3,721.37","4,967.23",0.00,0.00,0.00,0.72,0.28,0.12,0.25
4,6001,5/16/2025,"18,342,375.48","3,458.69","5,303.27",0.00,0.00,0.00,0.80,0.20,0.12,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...
704,6381,8/14/2025,"17,238,546.31","3,109.84","5,543.22",0.23,0.77,0.00,0.00,0.00,0.07,1.00
705,6381,9/15/2025,"17,911,357.93","3,178.83","5,634.58",0.25,0.75,0.00,0.00,0.00,0.07,1.00
706,6381,10/14/2025,"16,855,080.31","2,898.42","5,815.26",0.26,0.74,0.00,0.00,0.00,0.07,1.00
707,6381,11/14/2025,"18,988,003.54","3,234.41","5,870.62",0.33,0.67,0.00,0.00,0.00,0.07,1.00


In [32]:
liab_seg_df = fund_map_df.merge(liab_seg_df, on='Fund_ID')
liab_seg_df

,Product,Series,Fund_ID,Fund_Name,Bbg_Idx,Opt_Type,Mat(Yrs),Maturity Date,Notional,Contracts,IdxLvl_StartDt,IUL EFLIC,IUL EFLOA,IUL Reinsured,MSO EFLIC,MSO EFLOA,Cap,Buffer
0,VUL,MSO2,6017,S&P 500 1 Year -10% Buffer,SPX Index,Call Spread,1,1/17/2025,"903,023.91",186.58,"4,839.81",0.00,0.00,0.00,0.75,0.25,0.15,0.10
1,VUL,MSO2,6017,S&P 500 1 Year -10% Buffer,SPX Index,Call Spread,1,2/21/2025,"375,138.08",74.94,"5,005.57",0.00,0.00,0.00,0.67,0.33,0.15,0.10
2,VUL,MSO2,6017,S&P 500 1 Year -10% Buffer,SPX Index,Call Spread,1,3/21/2025,"390,341.53",76.28,"5,117.09",0.00,0.00,0.00,0.39,0.61,0.18,0.10
3,VUL,MSO2,6017,S&P 500 1 Year -10% Buffer,SPX Index,Call Spread,1,4/17/2025,"304,796.85",61.36,"4,967.23",0.00,0.00,0.00,0.45,0.55,0.16,0.10
4,VUL,MSO2,6017,S&P 500 1 Year -10% Buffer,SPX Index,Call Spread,1,5/16/2025,"748,582.28",141.15,"5,303.27",0.00,0.00,0.00,0.66,0.34,0.14,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,IUL,???,6052,S&P 500 3 Year -100% Buffer,SPX Index,Call Spread,3,8/16/2027,"2,463,178.25",444.36,"5,543.22",0.00,0.00,1.00,0.00,0.00,0.28,1.00
705,IUL,???,6052,S&P 500 3 Year -100% Buffer,SPX Index,Call Spread,3,9/14/2027,"2,642,457.63",468.97,"5,634.58",0.00,0.00,1.00,0.00,0.00,0.28,1.00
706,IUL,???,6052,S&P 500 3 Year -100% Buffer,SPX Index,Call Spread,3,10/14/2027,"1,645,614.59",282.98,"5,815.26",0.00,0.00,1.00,0.00,0.00,0.28,1.00
707,IUL,???,6052,S&P 500 3 Year -100% Buffer,SPX Index,Call Spread,3,11/15/2027,"2,944,341.03",501.54,"5,870.62",0.00,0.00,1.00,0.00,0.00,0.28,1.00


In [10]:
liab_seg_df.columns

Index(['Product', 'Series', 'Fund_ID', 'Fund_Name', 'Bbg_Idx', 'Opt_Type',
       'Maturity Date', 'Notional', 'IUL EFLIC', 'IUL EFLOA', 'IUL Reinsured',
       'MSO EFLIC', 'MSO EFLOA', 'Cap', 'Buffer', 'Part'],
      dtype='object')

In [33]:
# Rename Items
liab_seg_df.rename(columns={'Maturity Date': 'Seg_EndDt'}, inplace=True)

In [34]:
liab_seg_file_processed = r'C:\Users\S0053071\Repos\Winterfell\Positions\20241231\Winterfell_LiabilitySegments_20241231.csv'
liab_seg_df.to_csv(liab_seg_file_processed, index=False)

0.0.254.128:0.0.186.248:0.0.83.255:0.0.254.142:0.0.24.73
